In [1]:
# 导入所需要的包
import os
import numpy as np
import pandas as pd


In [ ]:
flight_data = pd.read_csv('flighttimechange_final.csv')
all_airports = pd.read_csv('DirectoryList2.csv')


columns_from_weather = [
    'DATE','NAME','REPORT_TYPE','HourlyPrecipitation', 'HourlyRelativeHumidity', 'HourlySkyConditions',
    'HourlyStationPressure', 'HourlyVisibility', 'HourlyWetBulbTemperature',
    'HourlyWindGustSpeed', 'HourlyWindSpeed','DailySnowDepth','DailySnowfall']

columns_drop_from_flight = ['QUARTER' , 'FL_DATE' , 'SCH_OP_CARRIER' , 'SCH_OP_CARRIER_FL_NUM' , 'OP_CARRIER' ,
                            'ORIGIN_AIRPORT_ID' , 'ORIGIN_AIRPORT_SEQ_ID' , 'ORIGIN_CITY_MARKET_ID' ,
                       'ORIGIN_STATE_ABR' , 'ORIGIN_STATE_NM', 'DEST_AIRPORT_ID' , 'DEST_AIRPORT_SEQ_ID' , 'DEST_CITY_MARKET_ID' ,
                       'DEST_STATE_ABR' , 'DEST_STATE_NM' , 
                       'DEP_TIME_BLK' , 'WHEELS_OFF' , 'WHEELS_ON' , 'ARR_TIME_BLK' ,
                       'AIR_TIME' , 'FLIGHTS' , 'CRS_ELAPSED_TIME','CANCELLATION_CODE']


flight_data = flight_data.drop(columns_drop_from_flight, axis=1)

# diverted is not considered
flight_data = flight_data[flight_data['DIVERTED']==0]
# not cancelled but no arrival time - one row
flight_data = flight_data.drop(flight_data[((flight_data['CANCELLED'] == 0) & (flight_data['ARR_TIME'].isna()))].index)
# not cancelled, delayed, but no delay reason
flight_data = flight_data.drop(flight_data[(flight_data['CANCELLED']==0) & (flight_data['ARR_DELAY'] > 30) & (flight_data['CARRIER_DELAY'].isna())].index)


flight_data['FL_DATE'] = pd.to_datetime(flight_data['Timechange_FL_DATE'])
# flight_data['FL_TIME'] = pd.to_datetime(flight_data['CRS_DEP_TIME'], format='%H%M', errors='coerce').dt.time

flight_data['FL_TIME'] = flight_data['Timechange_CRS_DEP_Time']



In [ ]:
# combine departure airport weather at scheduled departure time


combined_pt1 = pd.DataFrame()
manual_handle = []



for i,airport in all_airports.iterrows():
    print(i)
    if pd.isna(airport['ReplaceID']):
        StationID = airport['StationID']
    else:
        StationID = airport['ReplaceID']

    airportCode = airport['iata_code']
    weather_data = pd.read_csv(os.path.join('Weather_DATA_IN_CST_filled2',StationID+'.csv'), usecols=columns_from_weather)
    weather_data = weather_data.loc[(weather_data['REPORT_TYPE']!='SOD') & (weather_data['REPORT_TYPE']!='SOM')]
    weather_data['DATE'] = pd.to_datetime(weather_data['DATE'])
    weather_data['timestamp'] = weather_data['DATE']

    # Weather_origin
    weather_data_orig = weather_data.copy()
    weather_data_orig.columns = [col+'_ORIGIN' for col in list(weather_data_orig.columns)]
    weather_data_orig['timestamp'] = weather_data_orig['timestamp_ORIGIN']



    flight_by_origin = flight_data[flight_data['ORIGIN'] == airportCode]
#     flight_by_destination = flight_data[flight_data['DEST'] == airportCode]
    flight_by_origin['timestamp'] = pd.to_datetime(flight_by_origin['FL_DATE'].astype(str) + ' ' + flight_by_origin['FL_TIME'].astype(str), errors='coerce')
#     flight_by_destination['timestamp'] = pd.to_datetime(flight_by_destination['FL_DATE'].astype(str) + ' ' + flight_by_destination['FL_TIME'].astype(str), errors='coerce')


    # 处理flight NA值
    if len(flight_by_origin[flight_by_origin['timestamp'].isna()]) > 0:
        manual_handle.append([manual_handle, flight_by_origin[flight_by_origin['timestamp'].isna()]])
        flight_by_origin.drop(flight_by_origin[flight_by_origin['timestamp'].isna()].index, inplace=True)

   
        
    flight_by_origin['index'] = flight_by_origin.index.copy()
    
    
    combined_origin = pd.merge_asof(flight_by_origin.sort_values('timestamp'), 
                                  weather_data_orig.sort_values('timestamp'), 
                                  on='timestamp', 
                                  direction='nearest')
    
    combined_origin = combined_origin.reset_index().set_index('index').drop('level_0',axis=1)

    combined_origin = combined_origin[[col+'_ORIGIN' for col in columns_from_weather]]
    combined_pt1 = pd.concat([combined_pt1, flight_data.merge(combined_origin,left_index=True, right_index=True, how='inner')])


combined_pt1



In [ ]:
combined_pt1.to_csv('flight_weather_V9_Pt1.csv')

In [ ]:
# combine arrival airport weather at scheduled departure time

combined_pt2 = pd.DataFrame()
manual_handle = []



for i,airport in all_airports.iterrows():
    print(i)
    if pd.isna(airport['ReplaceID']):
        StationID = airport['StationID']
    else:
        StationID = airport['ReplaceID']
    airportCode = airport['iata_code']
    
    weather_data = pd.read_csv(os.path.join('Weather_DATA_IN_CST_filled2',StationID+'.csv'), usecols=columns_from_weather)
    weather_data = weather_data.loc[(weather_data['REPORT_TYPE']!='SOD') & (weather_data['REPORT_TYPE']!='SOM')]
    weather_data['DATE'] = pd.to_datetime(weather_data['DATE'])
    weather_data['timestamp'] = weather_data['DATE']

    # Weather_dest
    weather_data_dest = weather_data.copy()
    weather_data_dest.columns = [col+'_DEST' for col in list(weather_data_dest.columns)]
    weather_data_dest['timestamp'] = weather_data_dest['timestamp_DEST']



    flight_by_dest = combined_pt1[combined_pt1['DEST'] == airportCode]
#     flight_by_destination = flight_data[flight_data['DEST'] == airportCode]
    flight_by_dest['timestamp'] = pd.to_datetime(flight_by_dest['FL_DATE'].astype(str) + ' ' + flight_by_dest['FL_TIME'].astype(str), errors='coerce')
#     flight_by_destination['timestamp'] = pd.to_datetime(flight_by_destination['FL_DATE'].astype(str) + ' ' + flight_by_destination['FL_TIME'].astype(str), errors='coerce')


    # 处理flight NA值
    if len(flight_by_dest[flight_by_dest['timestamp'].isna()]) > 0:
        manual_handle.append([manual_handle, flight_by_dest[flight_by_dest['timestamp'].isna()]])
        flight_by_dest.drop(flight_by_dest[flight_by_dest['timestamp'].isna()].index, inplace=True)

   
        
    flight_by_dest['index'] = flight_by_dest.index.copy()
    
    
    combined_dest = pd.merge_asof(flight_by_dest.sort_values('timestamp'), 
                                  weather_data_dest.sort_values('timestamp'), 
                                  on='timestamp', 
                                  direction='nearest')
    
    combined_dest = combined_dest.reset_index().set_index('index').drop('level_0',axis=1)
    combined_dest = combined_dest[[col+'_DEST' for col in columns_from_weather]]
    combined_pt2 = pd.concat([combined_pt2, combined_pt1.merge(combined_dest,left_index=True, right_index=True, how='inner')])

    
#                                pd.concat([combined_origin, combined_dest])])
combined_pt2

In [ ]:
flight_V9 = combined_pt2.copy()

In [ ]:
flight_V9.to_csv('flight_weather_V9.csv')

In [ ]:
# FILTER OUT MISMATCHED DATES (>12HR)


file_path = "flight_weather_V9_1.csv"
output_path = "V9_1_filtered.csv"

# 定义chunksize以分块读取数据
chunksize = 10000  # 可根据内存情况调整块大小
first_chunk = True  # 标记是否是第一个块，以便写入表头

# 逐块读取并处理数据
for chunk in pd.read_csv(file_path, chunksize=chunksize):
    # 将 DATE_ORIGIN 和 DATE_DEST 列转换为 datetime 格式
    chunk['DATE_ORIGIN'] = pd.to_datetime(chunk['DATE_ORIGIN'], errors='coerce')
    chunk['DATE_DEST'] = pd.to_datetime(chunk['DATE_DEST'], errors='coerce')
    
    # 合并 Timechange_FL_DATE 和 Timechange_CRS_DEP 列为一个 datetime 列
    chunk['COMBINED_DATETIME'] = pd.to_datetime(
        chunk['Timechange_FL_DATE'].astype(str) + ' ' + chunk['Timechange_CRS_DEP_Time'].astype(str),
        errors='coerce'
    )
    
    # 计算与 DATE_ORIGIN 的时间差（以小时为单位）
    chunk['time_diff_origin'] = (chunk['COMBINED_DATETIME'] - chunk['DATE_ORIGIN']).abs() / pd.Timedelta(hours=1)
    
    # 计算与 DATE_DEST 的时间差（以小时为单位）
    chunk['time_diff_dest'] = (chunk['COMBINED_DATETIME'] - chunk['DATE_DEST']).abs() / pd.Timedelta(hours=1)

    # 筛选时间差超过 12 小时的记录，并标记是 DATE_ORIGIN 还是 DATE_DEST 问题
    filtered_chunk = chunk[(chunk['time_diff_origin'] > 12) | (chunk['time_diff_dest'] > 12)].copy()
#     filtered_chunk['Exceeds_12_Hours'] = filtered_chunk.apply(
#         lambda x: 'DATE_ORIGIN' if x['time_diff_origin'] > 12 else 'DATE_DEST',
#         axis=1
#     )

    # 将筛选出的数据追加写入CSV文件，只有第一个块写入表头
    filtered_chunk.to_csv(output_path, mode='w' if first_chunk else 'a', index=False, header=first_chunk)
    first_chunk = False  # 后续块不写入表头

print(f"时间差超过 12 小时的记录已逐块保存至 {output_path}")

In [2]:
flight_V9_1 = pd.read_csv('flight_weather_V9_1.csv')

/Users/shixin/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (44,47,48,49,51,53,57,60,61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# ADDED Y COLUMN
def time_to_minutes(time_str):
    """将时间字符串转换为当天的分钟数。如果时间为空，则返回 NaN。"""
    if time_str == 'Invalid Time':
        return 0
    hours, minutes = map(int, time_str.split(':'))
    return hours * 60 + minutes

flight_V9_1['ARR_Time_min'] = flight_V9_1['Timechange_ARR_Time'].apply(time_to_minutes)
flight_V9_1['DEP_Time_min'] = flight_V9_1['Timechange_CRS_DEP_Time'].apply(time_to_minutes)

# 处理跨日情况
flight_V9_1['Y'] = np.where(flight_V9_1['ARR_Time_min'] < flight_V9_1['DEP_Time_min'],
                   flight_V9_1['ARR_Time_min'] + 1440 - flight_V9_1['DEP_Time_min'],  # 到达次日
                   flight_V9_1['ARR_Time_min'] - flight_V9_1['DEP_Time_min'])          # 到达当天



In [4]:
# ADDED DELAYED COLUMN
flight_V9_1['DELAYED'] = flight_V9_1['ARR_DELAY'].apply(lambda x: 1 if x > 15 else 0)


In [7]:
# DEFINE DAYS AFTER HOLIDAY COLUMNS

from datetime import datetime

# 将日期列转换为日期格式
flight_V9_1['Timechange_FL_DATE'] = pd.to_datetime(flight_V9_1['Timechange_FL_DATE'], errors='coerce')

# 提取年份，并处理1月的日期
flight_V9_1['Year'] = flight_V9_1['Timechange_FL_DATE'].dt.year
flight_V9_1.loc[flight_V9_1['Timechange_FL_DATE'].dt.month == 1, 'Year'] -= 1

# 定义节假日日期并转换成 DataFrame 格式
holiday_data = {
    'Year': list(range(2017, 2024)),
    'Thanksgiving': [datetime(year, 11, 23) if year == 2017 else datetime(year, 11, 22) if year == 2018 else 
                     datetime(year, 11, 28) if year == 2019 else datetime(year, 11, 26) if year == 2020 else 
                     datetime(year, 11, 25) if year == 2021 else datetime(year, 11, 24) if year == 2022 else 
                     datetime(year, 11, 23) for year in range(2017, 2024)],
    'Christmas': [datetime(year, 12, 25) for year in range(2017, 2024)],
    'New Year': [datetime(year + 1, 1, 1) for year in range(2017, 2024)]
}
holiday_df = pd.DataFrame(holiday_data)

# 将节假日数据合并到原始数据上
flight_V9_1 = flight_V9_1.merge(holiday_df, on='Year', how='left')

# 计算每个节假日的天数差异
flight_V9_1['Days_after_Thanksgiving'] = (flight_V9_1['Timechange_FL_DATE'] - flight_V9_1['Thanksgiving']).dt.days
flight_V9_1['Days_after_Christmas'] = (flight_V9_1['Timechange_FL_DATE'] - flight_V9_1['Christmas']).dt.days
flight_V9_1['Days_after_New_Year'] = (flight_V9_1['Timechange_FL_DATE'] - flight_V9_1['New Year']).dt.days

# 删除不需要的列
flight_V9_1.drop(columns=['Year', 'Thanksgiving', 'Christmas', 'New Year','FL_DATE','FL_TIME'], inplace=True)

In [15]:
# Identify columns with mixed data types (float and str)

mixed_type_columns = []
for col in flight_V9_1.columns:
    # Check if the column has both float and str types
    unique_types = set(type(val) for val in flight_V9_1[col].dropna())
    if float in unique_types and str in unique_types:
        mixed_type_columns.append(col)
        print(f"Column '{col}' has mixed types: {unique_types}")

# Output columns with mixed types
if mixed_type_columns:
    print("\nColumns with mixed data types (float and str):")
    print(mixed_type_columns)
else:
    print("No columns with mixed data types (float and str) found.")

No columns with mixed data types (float and str) found.


In [11]:
# Function to clean the VISIBILITY values
def clean_visibility(value):
    if isinstance(value, str):
        # If the value ends with 'V' or 's', remove the last character
        if value.endswith('V') or value.endswith('s'):
            value = value[:-1]
        # If the value is '*', consider it as missing (NaN)
        elif value == '*':
            return np.nan
    # Convert the cleaned value to float if possible
    try:
        return float(value)
    except ValueError:
        return np.nan  # Return NaN if conversion to float fails

# Apply the cleaning function to the column
flight_V9_1['HourlyVisibility_ORIGIN'] = flight_V9_1['HourlyVisibility_ORIGIN'].apply(clean_visibility)
flight_V9_1['HourlyVisibility_DEST'] = flight_V9_1['HourlyVisibility_DEST'].apply(clean_visibility)

In [12]:
# Function to clean the WINDSPEED values
def clean_windspeed(value):
    if isinstance(value, str):
        # If the value ends with 'V' or 's', remove the last character
        if value.endswith('s'):
            value = value[:-1]
    # Convert the cleaned value to float if possible
    try:
        return float(value)
    except ValueError:
        return np.nan  # Return NaN if conversion to float fails

# Apply the cleaning function to the column
flight_V9_1['HourlyWindSpeed_ORIGIN'] = flight_V9_1['HourlyWindSpeed_ORIGIN'].apply(clean_windspeed)
flight_V9_1['HourlyWindSpeed_DEST'] = flight_V9_1['HourlyWindSpeed_DEST'].apply(clean_windspeed)

In [13]:
# Function to clean the PRECIPITATION values
def clean_precipitation(value):
    if isinstance(value, str):
        # If the value ends with 'V' or 's', remove the last character
        if value=='T':
            value = 0.01
    # Convert the cleaned value to float if possible
    try:
        return float(value)
    except ValueError:
        return np.nan  # Return NaN if conversion to float fails

# Apply the cleaning function to the column
flight_V9_1['HourlyPrecipitation_ORIGIN'] = flight_V9_1['HourlyPrecipitation_ORIGIN'].apply(clean_precipitation)
flight_V9_1['HourlyPrecipitation_DEST'] = flight_V9_1['HourlyPrecipitation_DEST'].apply(clean_precipitation)
flight_V9_1['DailySnowDepth_DEST'] = flight_V9_1['DailySnowDepth_DEST'].apply(clean_precipitation)
flight_V9_1['DailySnowfall_DEST'] = flight_V9_1['DailySnowfall_DEST'].apply(clean_precipitation)
flight_V9_1['DailySnowDepth_ORIGIN'] = flight_V9_1['DailySnowDepth_ORIGIN'].apply(clean_precipitation)
flight_V9_1['DailySnowfall_ORIGIN'] = flight_V9_1['DailySnowfall_ORIGIN'].apply(clean_precipitation)


In [38]:
# DEFINE EXTREME FOR ORIGIN AND DESTINATION
# 定义极端天气的条件




extreme_weather_conditions_ORIGIN = (
    (flight_V9_1['HourlyWindSpeed_ORIGIN'] > 13.9) |                    # 风速 > 13.9 m/s
    (flight_V9_1['HourlyWindGustSpeed_ORIGIN'] > 11.1) |                # 风阵 > 11.1 m/s
    ((flight_V9_1['HourlyRelativeHumidity_ORIGIN'] > 90) &              # 湿度 > 90% 且
     (flight_V9_1['HourlyVisibility_ORIGIN'] < 2)) |                    # 能见度 < 2 km
    (flight_V9_1['HourlyVisibility_ORIGIN'] < 2)| 
    (flight_V9_1['HourlyWetBulbTemperature_ORIGIN'] > 30) |             # 湿球温度 > 30°C
    (flight_V9_1['HourlyWetBulbTemperature_ORIGIN'] < -15) |            # 湿球温度 < -15°C
    (flight_V9_1['HourlyPrecipitation_ORIGIN'] > 10) |
    (flight_V9_1['HourlySkyConditions_ORIGIN'].str.contains('BKN|OVC|VV', na=False))  # 云层状况包括BKN, OVC等
)

extreme_weather_conditions_DEST = (
    (flight_V9_1['HourlyWindSpeed_DEST'] > 13.9) |                    # 风速 > 13.9 m/s
    (flight_V9_1['HourlyWindGustSpeed_DEST'] > 11.1) |                # 风阵 > 11.1 m/s
    ((flight_V9_1['HourlyRelativeHumidity_DEST'] > 90) &              # 湿度 > 90% 且
     (flight_V9_1['HourlyVisibility_DEST'] < 2)) |                    # 能见度 < 2 km
    (flight_V9_1['HourlyVisibility_DEST'] < 2)| 
    (flight_V9_1['HourlyWetBulbTemperature_DEST'] > 30) |             # 湿球温度 > 30°C
    (flight_V9_1['HourlyWetBulbTemperature_DEST'] < -15) |            # 湿球温度 < -15°C
    (flight_V9_1['HourlyPrecipitation_DEST'] > 10) |
    (flight_V9_1['HourlySkyConditions_DEST'].str.contains('BKN|OVC|VV', na=False))  # 云层状况包括BKN, OVC等
)





flight_V9_1['EXTREME_ORIGIN'] = extreme_weather_conditions_ORIGIN.astype(int)
flight_V9_1['EXTREME_DEST'] = extreme_weather_conditions_DEST.astype(int)

In [41]:
# 各变量的缺失值数量
flight_V9_1.isnull().sum().to_csv('V9_2_na.csv')

In [42]:
# 输出至CSV文件
output_path = "flight_weather_data_V9_2.csv"
flight_V9_1.to_csv(output_path, index=True)

print(f"符合极端天气条件的数据已保存至 {output_path}")

符合极端天气条件的数据已保存至 flight_weather_data_V9_2.csv


In [44]:
flight_V9_3 = pd.read_csv('flight_weather_data_V9_3.csv')

/Users/shixin/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (50) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [47]:
flight_V9_3[flight_V9_3['HourlyRelativeHumidity_DEST'].isna()].to_csv('V9_3_humidityMissing.csv')

In [45]:
flight_V9_3.columns

Index(['Unnamed: 0', 'index', 'YEAR', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK',
       'MKT_CARRIER', 'MKT_CARRIER_FL_NUM', 'OP_UNIQUE_CARRIER',
       'OP_CARRIER_AIRLINE_ID', 'TAIL_NUM', 'OP_CARRIER_FL_NUM', 'ORIGIN',
       'ORIGIN_CITY_NAME', 'DEST', 'DEST_CITY_NAME', 'CRS_DEP_TIME',
       'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'TAXI_IN', 'CRS_ARR_TIME',
       'ARR_TIME', 'ARR_DELAY', 'CANCELLED', 'DIVERTED', 'ACTUAL_ELAPSED_TIME',
       'DISTANCE', 'CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY',
       'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY', 'Timechange_CRS_DEP_Time',
       'Timechange_FL_DATE', 'Timechange_ARR_Time', 'DATE_ORIGIN',
       'NAME_ORIGIN', 'REPORT_TYPE_ORIGIN', 'HourlyPrecipitation_ORIGIN',
       'HourlyRelativeHumidity_ORIGIN', 'HourlySkyConditions_ORIGIN',
       'HourlyStationPressure_ORIGIN', 'HourlyVisibility_ORIGIN',
       'HourlyWetBulbTemperature_ORIGIN', 'HourlyWindGustSpeed_ORIGIN',
       'HourlyWindSpeed_ORIGIN', 'DailySnowDepth_ORIGIN',
       'DailyS

In [ ]:

# # 加载数据
# data = df

# # 将日期列转换为日期格式
# data['Timechange_FL_DATE'] = pd.to_datetime(data['Timechange_FL_DATE'], errors='coerce')

# # 提取年份，并处理1月的日期
# data['Year'] = data['Timechange_FL_DATE'].dt.year
# data.loc[data['Timechange_FL_DATE'].dt.month == 1, 'Year'] -= 1

# # 定义节假日日期并转换成 DataFrame 格式
# holiday_data = {
#     'Year': list(range(2017, 2024)),
#     'Thanksgiving': [datetime(year, 11, 23) if year == 2017 else datetime(year, 11, 22) if year == 2018 else 
#                      datetime(year, 11, 28) if year == 2019 else datetime(year, 11, 26) if year == 2020 else 
#                      datetime(year, 11, 25) if year == 2021 else datetime(year, 11, 24) if year == 2022 else 
#                      datetime(year, 11, 23) for year in range(2017, 2024)],
#     'Christmas': [datetime(year, 12, 25) for year in range(2017, 2024)],
#     'New Year': [datetime(year + 1, 1, 1) for year in range(2017, 2024)]
# }
# holiday_df = pd.DataFrame(holiday_data)

# # 将节假日数据合并到原始数据上
# data = data.merge(holiday_df, on='Year', how='left')

# # 计算每个节假日的天数差异
# data['Days_after_Thanksgiving'] = (data['Timechange_FL_DATE'] - data['Thanksgiving']).dt.days
# data['Days_after_Christmas'] = (data['Timechange_FL_DATE'] - data['Christmas']).dt.days
# data['Days_after_New_Year'] = (data['Timechange_FL_DATE'] - data['New Year']).dt.days

# # 删除不需要的列
# data.drop(columns=['Year', 'Thanksgiving', 'Christmas', 'New Year'], inplace=True)

In [ ]:
# # Identify columns with mixed data types (float and str)
# df = data
# mixed_type_columns = []
# for col in df.columns:
#     # Check if the column has both float and str types
#     unique_types = set(type(val) for val in df[col].dropna())
#     if float in unique_types and str in unique_types:
#         mixed_type_columns.append(col)
#         print(f"Column '{col}' has mixed types: {unique_types}")

# # Output columns with mixed types
# if mixed_type_columns:
#     print("\nColumns with mixed data types (float and str):")
#     print(mixed_type_columns)
# else:
#     print("No columns with mixed data types (float and str) found.")

In [ ]:
# # Function to clean the values
# def clean_visibility(value):
#     if isinstance(value, str):
#         # If the value ends with 'V' or 's', remove the last character
#         if value.endswith('V') or value.endswith('s'):
#             value = value[:-1]
#         # If the value is '*', consider it as missing (NaN)
#         elif value == '*':
#             return np.nan
#     # Convert the cleaned value to float if possible
#     try:
#         return float(value)
#     except ValueError:
#         return np.nan  # Return NaN if conversion to float fails

# # Apply the cleaning function to the column
# data['HourlyVisibility_ORIGIN'] = data['HourlyVisibility_ORIGIN'].apply(clean_visibility)
# data['HourlyVisibility_DEST'] = data['HourlyVisibility_DEST'].apply(clean_visibility)

In [ ]:
# # Function to clean the values
# def clean_windspeed(value):
#     if isinstance(value, str):
#         # If the value ends with 'V' or 's', remove the last character
#         if value.endswith('s'):
#             value = value[:-1]
#     # Convert the cleaned value to float if possible
#     try:
#         return float(value)
#     except ValueError:
#         return np.nan  # Return NaN if conversion to float fails

# # Apply the cleaning function to the column
# data['HourlyWindSpeed_ORIGIN'] = data['HourlyWindSpeed_ORIGIN'].apply(clean_windspeed)
# data['HourlyWindSpeed_DEST'] = data['HourlyWindSpeed_DEST'].apply(clean_windspeed)

In [ ]:
# # Function to clean the values
# def clean_precipitation(value):
#     if isinstance(value, str):
#         # If the value ends with 'V' or 's', remove the last character
#         if value=='T':
#             value = 0.01
#     # Convert the cleaned value to float if possible
#     try:
#         return float(value)
#     except ValueError:
#         return np.nan  # Return NaN if conversion to float fails

# # Apply the cleaning function to the column
# data['HourlyPrecipitation_DEST'] = data['HourlyPrecipitation_DEST'].apply(clean_precipitation)

In [ ]:
# # 定义极端天气的条件
# extreme_weather_conditions = (
#     (data['HourlyWindSpeed_DEST'] > 13.9) |                    # 风速 > 13.9 m/s
#     (data['HourlyWindGustSpeed_DEST'] > 11.1) |                # 风阵 > 11.1 m/s
#     ((data['HourlyRelativeHumidity_DEST'] > 90) &              # 湿度 > 90% 且
#      (data['HourlyVisibility_DEST'] < 2)) |                    # 能见度 < 2 km
#     (data['HourlyVisibility_DEST'] < 2)| 
#     (data['HourlyWetBulbTemperature_DEST'] > 30) |             # 湿球温度 > 30°C
#     (data['HourlyWetBulbTemperature_DEST'] < -15) |            # 湿球温度 < -15°C
#     (data['HourlyPrecipitation_DEST'] > 10) |
#     (data['HourlySkyConditions_DEST'].str.contains('BKN|OVC|VV', na=False))  # 云层状况包括BKN, OVC等
# )

# # 新增一列“EXTREME_WEATHER"，满足条件为1，否则为0
# data['EXTREME_WEATHER'] = extreme_weather_conditions.astype(int)

In [ ]:
# data.info()